In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow as tf, matplotlib.pyplot as plt
import os

from tensorflow.keras import layers

## Data load

In [3]:
# os.chdir('/content/drive/MyDrive/Cephalogram')
# os.chdir('/content/drive/MyDrive/Cephalogram/workspace/feature_test')
# import ceph_config as cf

# our modules
import utils.ceph_config as cf

cf.RESOLUTION_CONFIG

32

In [4]:
model_save_path = os.path.join(os.getcwd(), 'output', 'kfold', f'{cf.RESOLUTION_CONFIG}', 'model')
os.makedirs(model_save_path, exist_ok=True)

image_feature = np.load(os.path.join(os.getcwd(), 'output', f'{cf.RESOLUTION_CONFIG}', 'feature', 'feature.npy'))
binary_array = np.load(os.path.join(os.getcwd(), 'output', f'{cf.RESOLUTION_CONFIG}', 'array', 'binary_array.npy'))
int_array = np.load(os.path.join(os.getcwd(), 'output', f'{cf.RESOLUTION_CONFIG}', 'array', 'int_array.npy'))

image_feature_2 = image_feature.astype('float32')
binary_array_2 = binary_array.astype('float32')
int_array_2 = int_array.astype('float32')


print(image_feature_2.shape)
print(binary_array_2.shape)
print(int_array_2.shape)

print(image_feature_2.dtype)
print(binary_array_2.dtype)
print(int_array_2.dtype)

(6928, 96)
(6928, 3)
(6928,)
float32
float32
float32


In [5]:
x_test = image_feature_2[4560:5760]
x_train_all = np.concatenate((image_feature_2[0:4560], image_feature_2[5760:]))

y_test = int_array_2[4560:5760]
y_train_all = np.concatenate((int_array_2[0:4560], int_array_2[5760:]))

y2_test = binary_array[4560:5760]
y2_train_all = np.concatenate((binary_array[0:4560], binary_array[5760:]))

In [6]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=20, shuffle=True, random_state=7)

In [7]:
for train_index, test_index in kf.split(x_train_all):
    x_train, x_test     = x_train_all[train_index], x_train_all[test_index]
    y_train, y_test     = y_train_all[train_index], y_train_all[test_index]
    y2_train, y2_test   = y2_train_all[train_index], y2_train_all[test_index]

### model 생성

In [8]:
# model 생성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def create_model():
    input = layers.Input(shape=(96,))
    x = layers.Flatten()(input)
    x = layers.Dense(48)(x)

    x = layers.Dense(8)(x)
    output = layers.Activation('softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    ######################################################

    model_2 = Sequential()
    model_2.add(Dense(128, activation='relu', input_shape = (None, 96)))
    model_2.add(Dropout(0.1))
    model_2.add(Dense(64, activation='relu'))
    model_2.add(Dropout(0.1))
    model_2.add(Dense(y2_train.shape[1], activation='sigmoid'))


    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model_2.compile(loss='binary_crossentropy', optimizer='adam')

    return model, model_2

In [9]:
# 전체 훈련
from sklearn.model_selection import train_test_split

batch_size  = 8
epochs      = 200

for i in range(10):
    x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, test_size=0.1, random_state=i)
    _, _, y2_train, y2_val = train_test_split(x_train_all, y2_train_all, test_size=0.1, random_state=i)

    model, model_2 = create_model()

    print('\033[46m' + f'{i+1}번째 multi class model 훈련' + '\033[0m')
    history_1 = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                    validation_data=[x_val, y_val])

    print('\033[46m' + f'{i+1}번째 multi label model 훈련' + '\033[0m')
    history_2 = model_2.fit(x_train, y2_train, batch_size=batch_size, epochs=epochs,
                    validation_data=[x_val, y2_val])


    model.save(os.path.join(model_save_path, f'int_model_{i}.keras'))
    model_2.save(os.path.join(model_save_path, f'binary_model_{i}.keras'))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
645/645 [==============================] - 2s 3ms/step - loss: 0.0138 - val_loss: 0.0156
Epoch 108/200
645/645 [==============================] - 2s 3ms/step - loss: 0.0165 - val_loss: 0.0229
Epoch 109/200
645/645 [==============================] - 2s 3ms/step - loss: 0.0122 - val_loss: 0.0159
Epoch 110/200
645/645 [==============================] - 2s 3ms/step - loss: 0.0177 - val_loss: 0.0159
Epoch 111/200
645/645 [==============================] - 2s 3ms/step - loss: 0.0123 - val_loss: 0.0231
Epoch 112/200
645/645 [==============================] - 2s 3ms/step - loss: 0.0106 - val_loss: 0.0376
Epoch 113/200
645/645 [==============================] - 2s 3ms/step - loss: 0.0135 - val_loss: 0.0256
Epoch 114/200
645/645 [==============================] - 2s 3ms/step - loss: 0.0146 - val_loss: 0.0229
Epoch 115/200
645/645 [==============================] - 2s 3ms/step - loss: 0.0137 - val_loss: 0.0150
Epoch 116/200
645/645 [============================

### 최종 model 성능 평가

In [10]:
prediction_1_out = []
prediction_2_out = []

for i in range(10):
    model_1 = tf.keras.models.load_model(os.path.join(model_save_path, f'int_model_{i}.keras'))
    model_2 = tf.keras.models.load_model(os.path.join(model_save_path, f'binary_model_{i}.keras'))

    prediction_1 = model_1.predict(x_test)
    prediction_2 = model_2.predict(x_test)

    pred_class_1 = []

    for i, item in enumerate(prediction_1):
        pred_class_1.append(np.argmax(item))


    ########################
    pred_class_3 = []
    pred_class_2 = np.where(prediction_2 > 0.5, 1 , 0)

    for i, item in enumerate(pred_class_2):
        if item[0]==0 and item[1]==0 and item[2]==0:
            pred_class_3.append(0)

        elif item[0]==0 and item[1]==0 and item[2]==1:
            pred_class_3.append(1)

        elif item[0]==0 and item[1]==1 and item[2]==0:
            pred_class_3.append(2)

        elif item[0]==0 and item[1]==1 and item[2]==1:
            pred_class_3.append(3)

        elif item[0]==1 and item[1]==0 and item[2]==0:
            pred_class_3.append(4)

        elif item[0]==1 and item[1]==0 and item[2]==1:
            pred_class_3.append(5)

        elif item[0]==1 and item[1]==1 and item[2]==0:
            pred_class_3.append(6)

        elif item[0]==1 and item[1]==1 and item[2]==1:
            pred_class_3.append(7)

    prediction_1_out.append(np.array(pred_class_1).astype('float32'))
    prediction_2_out.append(np.array(pred_class_3).astype('float32'))

9/9 [==============================] - 0s 1ms/step


In [11]:
prediction_1_np = np.array(prediction_1_out)
prediction_2_np = np.array(prediction_2_out)

In [12]:
len(x_test)

286

In [13]:
len(prediction_1_np[0])

286

In [14]:
prediction_1_np

array([[5., 3., 2., ..., 6., 3., 0.],
       [5., 3., 2., ..., 4., 3., 0.],
       [5., 3., 2., ..., 6., 3., 0.],
       ...,
       [5., 3., 2., ..., 6., 3., 0.],
       [5., 3., 2., ..., 6., 3., 0.],
       [5., 3., 2., ..., 7., 3., 0.]], dtype=float32)

In [15]:
len(prediction_1_np)

10

In [16]:
output_multiclass = np.around(np.sum(prediction_1_np, axis=0) / len(prediction_1_np))
output_multilabel = np.around(np.sum(prediction_2_np, axis=0) / len(prediction_2_np))

In [17]:
print(y_test.shape)
print(output_multiclass.shape)
print(output_multilabel.shape)

(286,)
(286,)
(286,)


In [18]:
y_test

array([5., 3., 2., 1., 7., 1., 3., 4., 0., 2., 1., 0., 6., 5., 1., 6., 1.,
       4., 1., 5., 0., 2., 0., 5., 6., 5., 7., 3., 0., 1., 7., 6., 7., 5.,
       5., 7., 6., 3., 4., 4., 0., 3., 0., 3., 4., 2., 4., 2., 7., 3., 2.,
       6., 6., 6., 5., 1., 6., 3., 3., 4., 4., 5., 3., 1., 0., 1., 7., 6.,
       3., 4., 6., 0., 1., 6., 4., 5., 4., 2., 7., 0., 6., 7., 2., 4., 6.,
       4., 2., 0., 5., 5., 6., 7., 4., 2., 0., 7., 4., 3., 0., 3., 0., 6.,
       1., 4., 2., 2., 1., 1., 4., 6., 1., 5., 4., 6., 3., 7., 7., 0., 6.,
       6., 2., 4., 2., 4., 0., 6., 3., 4., 0., 4., 6., 1., 7., 5., 0., 3.,
       7., 4., 5., 5., 6., 2., 1., 3., 6., 7., 7., 0., 4., 5., 5., 3., 7.,
       0., 3., 4., 4., 3., 5., 0., 7., 4., 1., 2., 7., 4., 2., 2., 0., 2.,
       5., 4., 0., 7., 1., 2., 0., 3., 6., 5., 3., 0., 0., 5., 1., 0., 7.,
       1., 3., 6., 0., 5., 6., 1., 7., 2., 3., 4., 5., 0., 3., 2., 0., 1.,
       5., 7., 4., 5., 5., 5., 4., 1., 7., 7., 0., 3., 6., 7., 4., 2., 5.,
       3., 6., 1., 6., 1.

In [19]:
output_multiclass

array([5., 3., 2., 1., 7., 1., 3., 4., 0., 2., 1., 0., 6., 5., 1., 6., 1.,
       4., 1., 5., 0., 2., 0., 5., 6., 5., 7., 3., 0., 1., 7., 6., 7., 5.,
       5., 7., 6., 3., 4., 4., 0., 3., 0., 3., 4., 2., 4., 2., 7., 3., 2.,
       6., 6., 6., 5., 1., 6., 3., 3., 4., 4., 5., 3., 1., 0., 1., 7., 6.,
       3., 4., 6., 0., 1., 6., 4., 5., 4., 2., 7., 0., 6., 7., 2., 4., 6.,
       4., 2., 0., 5., 5., 6., 7., 4., 2., 0., 7., 4., 3., 0., 3., 0., 6.,
       1., 4., 2., 2., 1., 1., 4., 6., 1., 5., 4., 6., 3., 7., 7., 0., 6.,
       6., 2., 4., 2., 4., 0., 6., 3., 4., 0., 4., 6., 1., 7., 5., 0., 3.,
       7., 4., 5., 5., 6., 2., 1., 3., 6., 7., 7., 0., 4., 5., 5., 3., 7.,
       0., 3., 4., 4., 3., 5., 0., 7., 4., 1., 2., 7., 4., 2., 2., 0., 2.,
       5., 4., 0., 7., 1., 2., 0., 3., 6., 5., 3., 0., 0., 5., 1., 0., 7.,
       1., 3., 6., 0., 5., 6., 1., 7., 2., 3., 4., 5., 0., 3., 2., 0., 1.,
       5., 7., 4., 5., 5., 5., 4., 1., 7., 7., 0., 3., 6., 7., 4., 2., 5.,
       3., 6., 1., 6., 1.

In [20]:
output_multilabel

array([5., 3., 2., 1., 7., 1., 3., 4., 0., 2., 1., 0., 6., 5., 1., 6., 1.,
       4., 1., 5., 0., 2., 0., 5., 6., 5., 7., 3., 0., 1., 7., 6., 7., 5.,
       5., 7., 6., 3., 4., 4., 0., 3., 0., 3., 4., 2., 4., 2., 7., 3., 2.,
       6., 6., 6., 5., 1., 6., 3., 3., 4., 4., 5., 3., 1., 0., 1., 7., 6.,
       3., 4., 6., 0., 1., 6., 4., 5., 4., 2., 7., 0., 6., 7., 2., 4., 6.,
       4., 2., 0., 5., 5., 6., 7., 4., 2., 0., 7., 4., 3., 0., 3., 0., 6.,
       1., 4., 2., 2., 1., 1., 4., 6., 1., 5., 4., 6., 3., 7., 7., 0., 6.,
       6., 2., 4., 2., 4., 0., 6., 3., 4., 0., 4., 6., 1., 7., 5., 0., 3.,
       7., 4., 5., 5., 6., 2., 1., 3., 6., 7., 7., 0., 4., 5., 5., 3., 7.,
       0., 3., 4., 4., 3., 5., 0., 7., 4., 1., 2., 7., 4., 2., 2., 0., 2.,
       5., 4., 0., 7., 1., 2., 0., 3., 6., 5., 3., 0., 0., 5., 1., 0., 7.,
       1., 3., 6., 0., 5., 6., 1., 7., 2., 3., 4., 5., 0., 3., 2., 0., 1.,
       5., 7., 4., 5., 5., 5., 4., 1., 7., 7., 0., 3., 6., 7., 4., 2., 5.,
       3., 6., 1., 6., 1.

In [21]:
from sklearn.metrics import confusion_matrix, classification_report


print('# Multi class')
print('\n')
print(confusion_matrix(y_test, output_multiclass))
print(classification_report(y_test, output_multiclass))
print('\n\n')
print('# Multi label')
print('\n')
print(confusion_matrix(y_test, output_multilabel))
print(classification_report(y_test, output_multilabel))

# Multi class


[[38  0  0  0  0  0  0  0]
 [ 1 28  0  0  0  0  0  0]
 [ 0  0 28  0  0  0  0  0]
 [ 0  0  0 35  0  0  0  0]
 [ 0  0  0  0 39  0  0  0]
 [ 0  0  0  0  0 37  0  0]
 [ 0  0  0  0  0  0 41  2]
 [ 0  0  0  0  0  0  2 35]]
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99        38
         1.0       1.00      0.97      0.98        29
         2.0       1.00      1.00      1.00        28
         3.0       1.00      1.00      1.00        35
         4.0       1.00      1.00      1.00        39
         5.0       1.00      1.00      1.00        37
         6.0       0.95      0.95      0.95        43
         7.0       0.95      0.95      0.95        37

    accuracy                           0.98       286
   macro avg       0.98      0.98      0.98       286
weighted avg       0.98      0.98      0.98       286




# Multi label


[[38  0  0  0  0  0  0  0]
 [ 0 29  0  0  0  0  0  0]
 [ 0  0 28  0  0  0  0  0]
 [ 0  0  0 35  0